In [1]:
# Parameters
DB_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_element0\\interactions.db"
SEARCH_LEDGER_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_element0\\search_telemetry.json"


# Elements Playground

Use this notebook to inspect Elements graphs, pull definitions from the FastAPI `/api/elements` endpoints, and simulate a run locally before wiring the graph to Control Center.

In [2]:
GRAPH_PRESET = globals().get('GRAPH_PRESET', 'qa_loop')
GRAPH_ID = globals().get('GRAPH_ID', None)
ELEMENTS_API_URL = globals().get('ELEMENTS_API_URL', 'http://localhost:8000/api/elements')
OUTPUT_DIR = globals().get('OUTPUT_DIR', './_papermill')

In [3]:
from pathlib import Path
import json
import sys
from urllib import request, error


def _discover_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'PROJECT_OVERVIEW.md').exists():
            return candidate
    return start


REPO_ROOT = _discover_repo_root(Path.cwd().resolve())
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from datalab.scripts import elements as elements_utils

OUTPUT_DIR = Path(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
available_presets = elements_utils.available_presets()
available_presets

['insight_report', 'qa_loop']

In [4]:
def _fetch_graph(graph_id: str | None):
    if not graph_id:
        return elements_utils.load_graph(preset=GRAPH_PRESET)
    url = f"{ELEMENTS_API_URL.rstrip('/')}/graphs/{graph_id}"
    with request.urlopen(url, timeout=3.0) as response:
        return json.loads(response.read().decode('utf-8'))

try:
    graph_payload = _fetch_graph(GRAPH_ID)
except (error.URLError, OSError) as exc:
    graph_payload = elements_utils.load_graph(preset=GRAPH_PRESET)
    graph_payload.setdefault('metadata', {})['apiFallback'] = str(exc)

if 'tenantId' not in graph_payload and graph_payload.get('tenant_id'):
    graph_payload['tenantId'] = graph_payload.pop('tenant_id')
if 'workspaceId' not in graph_payload and graph_payload.get('workspace_id'):
    graph_payload['workspaceId'] = graph_payload.pop('workspace_id')
for edge in graph_payload.get('edges', []):
    if 'from' not in edge and edge.get('source'):
        edge['from'] = edge['source']
    if 'to' not in edge and edge.get('target'):
        edge['to'] = edge['target']

graph_payload

{'name': 'LLM QA Loop',
 'tenantId': 'lab',
 'workspaceId': 'default',
 'nodes': [{'id': 'node_prompt',
   'type': 'prompt',
   'label': 'Prompt',
   'props': {'text': 'Hello'}},
  {'id': 'node_llm',
   'type': 'llm',
   'label': 'LLM',
   'props': {'model': 'gpt-4o-mini', 'temperature': 0.1}},
  {'id': 'node_notebook',
   'type': 'notebook',
   'label': 'Notebook',
   'props': {'notebook': 'control_center_playground.ipynb'}}],
 'edges': [{'id': 'edge_prompt_llm',
   'from': {'node': 'node_prompt', 'port': 'text'},
   'to': {'node': 'node_llm', 'port': 'prompt'}},
  {'id': 'edge_llm_notebook',
   'from': {'node': 'node_llm', 'port': 'response'},
   'to': {'node': 'node_notebook', 'port': 'parameters'}}],
 'metadata': {'tags': ['qa'], 'createdBy': 'notebook'}}

In [5]:
graph_summary = elements_utils.graph_summary(graph_payload)
execution_plan = elements_utils.build_execution_plan(graph_payload)
graph_summary, execution_plan

({'name': 'LLM QA Loop',
  'tenantId': 'lab',
  'workspaceId': 'default',
  'node_count': 3,
  'edge_count': 2,
  'node_types': {'prompt': 1, 'llm': 1, 'notebook': 1},
  'tags': ['qa']},
 [{'step': 1, 'node': 'node_prompt', 'type': 'prompt', 'depends_on': []},
  {'step': 2,
   'node': 'node_llm',
   'type': 'llm',
   'depends_on': ['node_prompt']},
  {'step': 3,
   'node': 'node_notebook',
   'type': 'notebook',
   'depends_on': ['node_llm']}])

In [6]:
run_result = elements_utils.simulate_graph(graph_payload)
graph_slug = graph_summary['name'].lower().replace(' ', '_').replace('/', '-')
graph_path = elements_utils.write_graph(graph_payload, OUTPUT_DIR / f"{graph_slug}.json")
trace_path = elements_utils.export_trace(run_result['trace'], OUTPUT_DIR / f"{graph_slug}-trace.json")
run_result['artifacts'] = {'graph': str(graph_path), 'trace': str(trace_path)}
run_result

{'status': 'succeeded',
 'outputs': {'status': 'queued',
  'notebook': 'control_center_playground.ipynb',
  'parameters': {'inputs': {'parameters': '[gpt-4o-mini | temp=0.1] Hello'}}},
 'trace': [{'id': 'node_prompt',
   'type': 'prompt',
   'inputs': {},
   'outputs': {'text': 'Hello', 'variant': 'raw'},
   'props': {'text': 'Hello'}},
  {'id': 'node_llm',
   'type': 'llm',
   'inputs': {'prompt': 'Hello'},
   'outputs': {'response': '[gpt-4o-mini | temp=0.1] Hello',
    'model': 'gpt-4o-mini',
    'temperature': 0.1},
   'props': {'model': 'gpt-4o-mini', 'temperature': 0.1}},
  {'id': 'node_notebook',
   'type': 'notebook',
   'inputs': {'parameters': '[gpt-4o-mini | temp=0.1] Hello'},
   'outputs': {'status': 'queued',
    'notebook': 'control_center_playground.ipynb',
    'parameters': {'inputs': {'parameters': '[gpt-4o-mini | temp=0.1] Hello'}}},
   'props': {'notebook': 'control_center_playground.ipynb'}}],
 'artifacts': {'graph': '_papermill\\llm_qa_loop.json',
  'trace': '_pape

In [7]:
{
    "kernelspec": {
        "name": "python3",
        "display_name": "Python 3 (ipykernel)",
        "language": "python"
    },
    "language_info": {
        "name": "python",
        "version": "3.13.9",
        "mimetype": "text/x-python",
        "codemirror_mode": {
            "name": "ipython",
            "version": 3
        },
        "pygments_lexer": "ipython3",
        "nbconvert_exporter": "python",
        "file_extension": ".py"
    }
}

{'kernelspec': {'name': 'python3',
  'display_name': 'Python 3 (ipykernel)',
  'language': 'python'},
 'language_info': {'name': 'python',
  'version': '3.13.9',
  'mimetype': 'text/x-python',
  'codemirror_mode': {'name': 'ipython', 'version': 3},
  'pygments_lexer': 'ipython3',
  'nbconvert_exporter': 'python',
  'file_extension': '.py'}}